In [1]:
"""
Purpose of this table:
1) For all of the postysn segments WITH A TRACE (including soma and orphan)
computes the n_shared_segments and the n_shared_synapses as well as the unions

"""

'\nPurpose of this table:\n1) For all of the postysn segments WITH A TRACE (including soma and orphan)\ncomputes the n_shared_segments and the n_shared_synapses as well as the unions\n\n'

In [2]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import itertools 
import pandas as pd
from tqdm import tqdm
import seaborn as sns
from scipy import stats
import datajoint as dj
from collections import defaultdict
import pycircstat as pycs
from funconnect import morphology, connectomics, ta3, nda
from tqdm import tqdm
import scipy.stats as stats


Connecting celiib@10.28.0.34:3306


In [3]:
#setting the address and the username
dj.config['database.host'] = '10.28.0.34'
dj.config['database.user'] = 'celiib'
dj.config['database.password'] = 'newceliipass'
dj.config['safemode']=True
dj.config["display.limit"] = 20


pinky = dj.create_virtual_module('pinky', 'microns_pinky')
pinky_nda = dj.create_virtual_module('pinky_nda', 'microns_pinky_nda')
pinky_radtune = dj.create_virtual_module('pinky_radtune', 'microns_pinky_radtune')
pinky_spattune = dj.create_virtual_module('pinky_spattune', 'microns_pinky_spattune')
pinky_fc = dj.create_virtual_module('pinky_fc', 'microns_pinky_fc')


In [ ]:
#pinky.SharedInput() #did not exist

In [ ]:
from utils import find_pearson, find_cosine, find_binary_sim

schema = dj.schema('microns_pinky_fc', create_tables=True)

@schema
class SharedInput(dj.Computed):
    definition = """    
    -> pinky.Segment
    segment_b           : bigint   # segment id unique within each Segmentation
    ---
    n_syn_a     : int # number of synapses for cell A
    n_syn_b     : int # number of synapses for cell B
    n_syn_union : int # number of unique synapses for both
    n_syn_shared: int # number of shared synapses
    n_seg_a     : int # number of segments for cell A
    n_seg_b     : int # number of segments for cell B
    n_seg_union : int # number of unique segments for both
    n_seg_shared: int # number of shared segments
    """

    key_source = pinky.Segmentation & pinky.CurrentSegmentation

    def make(self, key):
        #gets all of the segments with a trace entry minus the segmentExclusdes (all the excitatory and orphans)
        targets = (pinky.Segment & pinky_nda.Trace & key) - pinky.SegmentExclude 
        #gets all of the axons and the synapses where these axons are the presyns
        axons = pinky.Neurite & 'neurite_type="axon"'
        synapse = (pinky.Synapse & axons.proj(presyn='segment_id') & key) - pinky.SynapseExclude
        
        #get all the combinations of postsyn targets without any repeats
        info = targets * targets.proj(segment_b='segment_id') & 'segment_id < segment_b'
        
        #gets all of the synapses where the "segment_id" is the postsyn
        A = (synapse & targets.proj(postsyn='segment_id')).proj(
            'presyn', syn1='synapse_id', segment_id='postsyn')
        
        #gets all of the synapses where the "segment_id" is the postsyn and name it segment_b
        #only difference between this table and the previous one is the names syn2 and segment_b
        B = (synapse * dj.U('presyn') & targets.proj(postsyn='segment_id')).proj(
            'presyn', syn2='synapse_id', segment_b='postsyn')
        """
        A * B & 'segment_id < segment_b': will do every combination of synapse from that presyn that is on a different segment
        Then for each segment pair
        n_syn_shared: total number of different synapse combinations from a common presyn for those to pairs
        n_seg_shared: number of distinct common presyns

        """
        shared = dj.U('segment_id', 'segment_b').aggr(A * B & 'segment_id < segment_b',
                                                      n_syn_shared='count(*)',
                                                      n_seg_shared='count(DISTINCT presyn)')
        
        
        #number of synapse from one all axons and distinct presyn axons that attach to it
        a = dj.U('segment_id').aggr(A, n_syn_a='count(*)', n_seg_a='count(DISTINCT presyn)') 
        #number of synapse from one presyn and distinct presyn axons that attach to it (BUT JUST WITH DIFFERENT NAMES)
        b = dj.U('segment_b').aggr(B, n_syn_b='count(*)', n_seg_b='count(DISTINCT presyn)')
        
        
        """
        ****This table restricts only to combinations of postsyns that have common presyn synapsing onto it
        1) starts with all info: all the combinations of postsyns without repeats
        2) appends on the n_syn_a that 1st segment gets from all axons
        3) appends on the n_syn_b that 2nd segment gets from all axons
        4) For that combination of segments adds on:
            a. total number of different synapse combinations from a common presyn for those to pairs
            b. total number of common presyns
        5) Then computes the following:
            a. n_syn_a + n_syn_b - (sum of all combinations of synapses from common presyns)
            --> so this could be negative hypothetically
            b. n_seg_a + n_seg_b - (total number of common presyns)
            --> aka the union of all presyns going to that pair


        """
        stats = (info * a * b * shared).proj('n_syn_a', 'n_syn_b', 'n_syn_shared',
                                             'n_seg_a', 'n_seg_b', 'n_seg_shared',
                                             n_syn_union='n_syn_a + n_syn_b - n_syn_shared',
                                             n_seg_union='n_seg_a + n_seg_b - n_seg_shared')
        
        self.insert([dict(key, **k) for k in tqdm(stats.fetch(as_dict=True))],
                    ignore_extra_fields=True)
        
        """
        For those combinations that don't have common presyns
        - just calculates the unions as the individual sums
        
        """
        non_shared = ((info - stats) * a * b).proj('n_syn_a', 'n_syn_b',
                                                    'n_seg_a', 'n_seg_b',
                                                    n_syn_union='n_syn_a + n_syn_b',
                                                    n_seg_union='n_seg_a + n_seg_b')
        self.insert([dict(key, **k, n_syn_shared=0, n_seg_shared=0) for k in tqdm(non_shared.fetch(as_dict=True))],
                    ignore_extra_fields=True)


In [ ]:
import time
start_time = time.time()
SharedInput.populate()
print(f"Total time = " + str(time.time() - start_time))

In [ ]:
key = pinky.Segmentation & pinky.CurrentSegmentation.fetch1()

In [ ]:
targets = (pinky.Segment & pinky_nda.Trace & key) - pinky.SegmentExclude 
targets

In [ ]:
(926*926 - 926)/2,len(info)

In [ ]:
axons = pinky.Neurite & 'neurite_type="axon"'
synapse = (pinky.Synapse & axons.proj(presyn='segment_id') & key) - pinky.SynapseExclude

In [ ]:
#get all the combinations of postsyn targets without any repeats
info = targets * targets.proj(segment_b='segment_id') & 'segment_id < segment_b'

A = (synapse & targets.proj(postsyn='segment_id')).proj(
    'presyn', syn1='synapse_id', segment_id='postsyn')

A

In [ ]:
#gets all of the synapses where the "segment_id" is the postsyn but says that segment_b is the postsyn
B = (synapse * dj.U('presyn') & targets.proj(postsyn='segment_id')).proj(
    'presyn', syn2='synapse_id', segment_b='postsyn')

B

In [ ]:
1744058/202500

In [ ]:
A * B

In [ ]:
"""
A * B & 'segment_id < segment_b': will do every combination of synapse from that presyn that is on a different segment
Then for each segment pair
n_syn_shared: number of different synapse combinations from a common presyn for those to pairs
n_seg_shared: number of distinct common presyns

"""
shared = dj.U('segment_id', 'segment_b').aggr(A * B & 'segment_id < segment_b',
                                                      n_syn_shared='count(*)',
                                                      n_seg_shared='count(DISTINCT presyn)')

In [ ]:
A

In [ ]:
#number of synapse from one presyn and distinct presyn axons that attach to it
a = dj.U('segment_id').aggr(A, n_syn_a='count(*)', n_seg_a='count(DISTINCT presyn)') 
#number of synapse from one presyn and distinct presyn axons that attach to it (BUT JUST WITH DIFFERENT NAMES)
b = dj.U('segment_b').aggr(B, n_syn_b='count(*)', n_seg_b='count(DISTINCT presyn)')

In [ ]:
a

In [ ]:
b